for pyomo:      conda install -c conda-forge ipopt glpk

In [1]:
import sys
sys.path.append('C://Users/Maxime/PycharmProjects/OpenIOCanada/')
import openIO
import pandas as pd
import numpy as np
import re
import os

from pyomo.environ import *

IOTables class will generate needed tables and currently stops right before dealing with provincial trade.

** Data available here https://drive.google.com/drive/u/1/folders/1Ba2tsPyyKw23ti7sSB6VyOw2BgOIGYxI **

In [2]:
folder_SUTs = 'C://Users/Maxime/Desktop/Thesis/Bdd/IO_Canada/Summary level/'
NPRI_excel = 'C://Users/Maxime/Desktop/Thesis/Bdd/IO_Canada/2016_INRP-NPRI.xlsx'

tables = openIO.IOTables(folder_SUTs, NPRI_excel, classification="industry")

Reading all the Excel files...
Formatting the Supply and Use tables...
Aggregating final demand sectors...
Removing IOIC codes from index...


C:\Users\Maxime\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Format the provincial trade Excel sheet into a pandas DataFrame

In [3]:
provincial_trade_file = 'C://Users/Maxime/Desktop/Thesis/Bdd/IO_Canada/Summary level/Provincial_trade_flow_2016_S.xlsx'

province_trade = pd.read_excel(provincial_trade_file,'Data')

province_trade.Origin = [{v: k for k, v in tables.matching_dict.items()}[i.split(') ')[1]] if (
            ')' in i and i != '(81) Canadian territorial enclaves abroad') else i for i in
                            province_trade.Origin]
province_trade.Destination = [{v: k for k, v in tables.matching_dict.items()}[i.split(') ')[1]] if (
            ')' in i and i != '(81) Canadian territorial enclaves abroad') else i for i in
                                 province_trade.Destination]
# extracting and formatting trade for each province
province_trade = pd.pivot_table(data=province_trade, index='Destination', columns=['Origin', 'Product'])

province_trade = province_trade.loc[
    [i for i in province_trade.index if i in tables.matching_dict], [i for i in province_trade.columns if
                                                                        i[1] in tables.matching_dict]]
province_trade *= 1000000
province_trade.columns = [(i[1], i[2].split(': ')[1]) if ':' in i[2] else i for i in
                             province_trade.columns]
province_trade.drop([i for i in province_trade.columns if i[1] not in [i[1] for i in tables.commodities]],
                       axis=1, inplace=True)
province_trade.columns = pd.MultiIndex.from_tuples(province_trade.columns)

Remove intraprovince trade

In [4]:
for province in province_trade.index:
    province_trade.loc[province, province] = 0
province_trade

AB  \
            Education services provided by government sector   
Destination                                                    
AB                                                       0.0   
BC                                                       0.0   
MB                                                       0.0   
NB                                                       0.0   
NL                                                       0.0   
NS                                                       0.0   
NT                                                       0.0   
NU                                                       0.0   
ON                                                       0.0   
PE                                                       0.0   
QC                                                       0.0   
SK                                                       0.0   
YT                                                       0.0   

                                                           \
            Health services provided by government sector   
Destination                                                 
AB                                                    0.0   
BC                                                    0.0   
MB                                                    0.0   
NB                                                    0.0   
NL                                                    0.0   
NS                                                    0.0   
NT                                                    0.0   
NU                                                    0.0   
ON                                                    0.0   
PE                                                    0.0   
QC                                                    0.0   
SK                                                    0.0   
YT                                                    0.0   

                                               \
            Other federal government services   
Destination                                     
AB                                        0.0   
BC                                        0.0   
MB                                        0.0   
NB                                        0.0   
NL                                        0.0   
NS                                        0.0   
NT                                        0.0   
NU                                        0.0   
ON                                        0.0   
PE                                        0.0   
QC                                        0.0   
SK                                        0.0   
YT                                        0.0   

                                                                  \
            Other provincial and territorial government services   
Destination                                                        
AB                                                         0.0     
BC                                                         0.0     
MB                                                         0.0     
NB                                                         0.0     
NL                                                         0.0     
NS                                                         0.0     
NT                                                         0.0     
NU                                                         0.0     
ON                                                         0.0     
PE                                                         0.0     
QC                                                         0.0     
SK                                                         0.0     
YT                                                         0.0     

                                                 \
            Other municipal government services   
Destination                                       
AB                           

Let's try to solve the provincial trade issue for the Alberta province first:

In [5]:
province = 'AB'

Define total domestic use (U*, which includes final demand too)

In [6]:
U_Y = pd.concat([tables.U.loc[province,province],tables.Y.loc[province,province]],axis=1)

Aggregate the province_trade data per commodity (disregarding the exporting province)

In [7]:
total_imports = province_trade.sum().loc[province]
index_commodity = [i[1] for i in tables.commodities]
total_imports = total_imports.reindex(index_commodity).fillna(0)

Our initial estimate for the distribution of interprovincial trade is:

In [8]:
initial_distribution = ((U_Y.T/(U_Y.sum(axis=1)))*total_imports).T

This initial estimates triggers negative values when trying to remove provincial imports from U

In [9]:
(U_Y - initial_distribution)[(U_Y - initial_distribution) < 0].dropna(how='all',axis=1).dropna(how='all')

,Support activities for agriculture and forestry,Crop and animal production,"Mining, quarrying, and oil and gas extraction",Utilities,Residential building construction,Non-residential building construction,Engineering construction,Repair construction,Other activities of the construction industry,Manufacturing,...,Government education services,Government health services,Other federal government services,Other provincial and territorial government services,Other municipal government services,Other aboriginal government services,Non-profit institutions serving households,Changes in inventories,Household final consumption expenditure,International exports
Live animals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-5.100062e+07,NaN,NaN
Forestry products and services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-8.152095e+05,NaN,NaN
Mineral fuels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.297612e+08,NaN,NaN
Non-metallic minerals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.365645e+07,NaN,NaN
Food and non-alcoholic beverages,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.237890e+08,NaN,NaN
Wood products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.264699e+07,NaN,NaN
Printed products and services,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.085904e+07,NaN,NaN
Refined petroleum products (except petrochemicals),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-4.233962e+07,NaN,NaN
Chemical products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.559496e+08,NaN,NaN
Plastic and rubber products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-3.785775e+07,NaN,NaN


Use pyomo to solve this optimization problem

In [10]:
opt = pyomo.environ.SolverFactory('ipopt')
model = ConcreteModel()

Drop our (fixed) parameters in our ConcreteModel

In [11]:
model.U0 = initial_distribution
model.UY = U_Y
model.imports = total_imports

model.products = model.UY.index
model.sectors = model.UY.columns

Define our variable (the optimized estimated, distributing provincial trade without creating negative values)

In [12]:
model.Ui = Var(initial_distribution.index, initial_distribution.columns, domain=NonNegativeReals, initialize=1)

The optimization must respect 2 constraints.

1) No negative values in our results when performing model.UY - model.Ui

In [13]:
def cons_positive(model, product, sector):
    return model.UY.loc[product, sector]-model.Ui[product, sector] >= 0

model.constraint_positive = Constraint(model.products, model.sectors, rule=cons_positive)

2) Sum of the values from our dataframe variable must equal total_imports values for each imported products

In [14]:
def cons_total(model, product):
    return sum(model.Ui[product, :]) ==  model.imports[product]

model.constraint_total = Constraint(model.products, rule=cons_total)

The objective function is to try to minimize the difference between model.Ui and model.U0

In [15]:
def obj_minimize(model):
    for sector in model.U0.columns:
        for product in model.U0.index:
            return ((model.U0.loc[product,sector]-model.Ui[product, sector])/(model.U0.loc[product,sector]+1e-10))**2

model.obj = Objective(rule=obj_minimize, sense=minimize)

Solving

In [16]:
sol = opt.solve(model)

    model=unknown;
        message from solver=Ipopt 3.11.1\x3a Converged to a locally infeasible
        point. Problem may be infeasible.


Changing the solution found into a dataframe

In [17]:
df = pd.DataFrame.from_dict(model.Ui.extract_values(),orient='index')
df['product'] = [i[0] for i in df.index]
df['sector'] = [i[1] for i in df.index]
df.columns = ['values', 'product', 'sector']
Ui = pd.pivot_table(data=df, index='product', columns='sector', values='values')

Check that there are no more negatives values when making the difference with U_Y

In [18]:
U_Y - Ui

,Accommodation and food services,"Administrative and support, waste management and remediation services","Advertising, promotion, meals, entertainment, and travel","Arts, entertainment and recreation",Changes in inventories,Crop and animal production,Educational services,Engineering construction,"Finance, insurance, real estate, rental and leasing and holding companies","Fishing, hunting and trapping",...,"Professional, scientific and technical services",Repair construction,"Repair, maintenance and operating and office supplies",Residential building construction,Retail trade,Support activities for agriculture and forestry,Transportation and warehousing,Transportation margins,Utilities,Wholesale trade
Accommodation and food services,6.438475e+07,2.400598e+08,-1.000060e+00,2.556475e+07,-1.000060e+00,-1.499600e-01,-2.147500e-01,-1.376400e-01,3.394428e+08,-3.000010e-04,...,4.543318e+08,-9.980000e-03,-1.000060e+00,-2.014800e-01,1.593758e+08,-1.481000e-02,2.591948e+08,-1.000060e+00,-7.389000e-02,8.266375e+07
"Administrative and support, head office, waste management and remediation services",3.018568e+08,8.120618e+08,-1.000060e+00,1.126708e+08,-1.000060e+00,-3.145865e-01,3.710751e+06,5.408248e+08,2.779553e+09,-2.100014e-04,...,1.555586e+09,3.890458e+08,-1.000060e+00,7.708475e+07,1.441907e+09,-2.245000e-02,1.021979e+09,-1.000060e+00,2.357938e+08,1.768252e+09
Alcoholic beverages and tobacco products,1.339638e+08,-9.375000e-02,-1.000060e+00,5.737511e+05,1.396475e+07,-5.720000e-03,-7.050000e-03,-8.140000e-03,-1.434000e-01,-1.000060e+00,...,-1.187900e-01,-7.800004e-04,-1.000060e+00,-1.639000e-02,-2.112400e-01,-6.300005e-04,-6.701000e-02,-1.000060e+00,-2.750000e-03,-1.037900e-01
"Arts, entertainment and recreation services",9.752722e+07,5.393922e+07,-1.000327e-08,1.596192e+08,-1.000327e-08,-1.000327e-08,-5.710000e-02,-1.000327e-08,1.097742e+08,-1.000327e-08,...,3.615822e+07,-1.000327e-08,-1.000327e-08,-1.000327e-08,-1.000327e-08,-1.000327e-08,-1.000327e-08,-1.000327e-08,-1.000327e-08,-1.000327e-08
Chemical products,-2.770703e-01,1.181918e+08,-1.000060e+00,-1.350400e-01,-2.909120e+08,1.459143e+09,-3.104000e-02,1.288708e+08,4.251775e+07,-3.400009e-04,...,8.911675e+07,2.713758e+08,-1.000060e+00,6.444075e+07,9.207511e+05,-1.641000e-02,1.631375e+07,-1.000060e+00,-5.765000e-02,4.537875e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Transportation margins,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,...,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337139e-21,-1.337435e-21
Utilities,1.132533e+08,3.953034e+07,-9.961788e-09,1.919934e+07,-9.961788e-09,2.108333e+08,1.664344e+06,2.315734e+07,3.915173e+08,-1.999996e-05,...,4.847934e+07,5.418344e+06,-9.961788e-09,5.678344e+06,1.989823e+08,1.387344e+06,2.548353e+08,-9.961788e-09,7.261344e+06,1.108103e+08
Wholesale margins and commissions,2.560558e+08,9.501575e+07,-1.000060e+00,1.718075e+07,-1.000060e+00,5.507598e+08,-4.075000e-02,1.791359e+09,1.118918e+08,-9.400003e-04,...,1.429138e+08,3.088688e+08,-1.000060e+00,7.468518e+08,7.020175e+07,-8.743000e-02,2.844798e+08,-1.000060e+00,3.086875e+07,1.272788e+08
Wood products,-9.380000e-03,-1.000060e+00,-1.000060e+00,-6.450000e-03,-3.445200e+07,-1.429400e-01,-1.000060e+00,-2.313100e-01,-1.000060e+00,-1.000060e+00,...,-1.000060e+00,7.388775e+07,-1.000060e+00,1.310864e+09,-1.330000e-03,-4.400007e-04,-2.901000e-02,-1.000060e+00,-2.533000e-02,-1.760600e-01


Check that total value of imports is kept

In [19]:
np.isclose(Ui.sum().sum(),total_imports.sum())

False

In [20]:
total_imports.sum()

63647054000.0

In [21]:
Ui.sum().sum()

19759301467.300537